In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from torch import nn, optim

In [31]:
#data = 'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230518_124958.csv'
data = r'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230518_124958.csv'
merge_data = pd.read_csv(data, encoding='ANSI')
#print(weather_data)
print(merge_data.head(1))

        mmsi ship_name  ship_type   
0  440051540      D-01          0  \

                                                geom    cog  sog   
0  0101000020110F00000000002039676B41000000002DCF...  329.2  5.7  \

        insert_time  지방청           표지                일시   풍향   유향    기온    수온   
0  2023-05-11 10:10  부산청  신항유도등부표(랜비)  2023-05-11 10:10  196  246  16.4  17.0  \

     풍속   유속    기압  습도  
0  9.77  0.3  1017  82  


In [32]:
# 데이터를 교육 및 테스트 세트로 분할
X = merge_data[["mmsi","ship_name","ship_type","cog","sog","insert_time","풍향","기온","수온","풍속","유속","기압","습도"]]  # 입력 값 (excluding "geom")
y = merge_data["geom"]  # 출력 값 (geom)

print(X)
#print(y)

            mmsi      ship_name  ship_type    cog   sog       insert_time   
0      440051540           D-01          0  329.2   5.7  2023-05-11 10:10  \
1      440300780            NaN          0  329.8   0.0  2023-05-11 10:10   
2      440208550            NaN          0  139.3   0.0  2023-05-11 10:10   
3      440275000  GBK EXPRESS 1         40  175.8   0.0  2023-05-11 10:10   
4      440414850   NO1 GEO SUNG         80    0.0   0.0  2023-05-11 10:10   
...          ...            ...        ...    ...   ...               ...   
16180  538010219  SAWASDEE VEGA         70  225.2  16.0  2023-05-11 10:29   
16181  440121850            NaN          0    0.0   0.0  2023-05-11 10:30   
16182  440110850  PILOT ORYUKDO         50  322.3   1.7  2023-05-11 10:30   
16183  440107840   NO.7 GEUMHWA         30    0.0   0.0  2023-05-11 10:30   
16184  440134620       HYUNJUNG         80  231.3   0.7  2023-05-11 10:30   

        풍향    기온    수온     풍속   유속    기압  습도  
0      196  16.4  17.0   9.7

In [33]:
encoded_ship_names = pd.get_dummies(merge_data["ship_name"], prefix="ship_name")
X = pd.concat([X, encoded_ship_names], axis=1).drop("ship_name", axis=1)

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=

0.2, random_state=42)

#학습용, 검증용, 시험용으로 분리
train_size = int(327 * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+33], y[train_size:train_size+33]
X_test, y_test = X[train_size+33:], y[train_size+33:]

In [28]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(2)) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=32)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_16084\1443939963.py", line 5, in <module>
    model.fit(X_train, y_train, epochs=100, batch_size=32)
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\framework\constant_op.py", line 103, in convert_to_eager_tensor
    return ops.EagerTensor(value, ctx.device_name, dtype)
ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Pyt